# Yield Curve Model Testing Notebook

This notebook exercises the `YieldCurve` class by:
1. Building a flat dummy curve for SOFR and USD LIBOR.
2. Computing spot discount factors for various tenors.
3. Computing forward rates for overnight and IBOR indices.

---

## 1) Import dependencies

We start by importing the necessary libraries and the `YieldCurve` class.

In [1]:
import sys, os

# compute the parent of tests → FixedIncomeLib
repo_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if repo_root not in sys.path:
    sys.path.insert(0, repo_root)

print("Added to sys.path:", repo_root)

Added to sys.path: c:\Users\neels\OneDrive\Desktop\Capstone_Project\FixedIncomeLib


In [2]:
import pandas as pd
import numpy as np
from yield_curve.yield_curve_model import YieldCurve
from data import DataCollection, Data1D


## 2) Define dummy curve data

Create a small DataFrame with dummy rates for two indices, at 1M, 3M, and 6M tenors.

In [3]:
DUMMY_DATA = [
    ['SOFR-1B',             '1M',  0.0020],
    ['SOFR-1B',             '3M',  0.0025],
    ['SOFR-1B',             '6M',  0.0030],
    ['USD-LIBOR-BBA-3M',    '1M',  0.0040],
    ['USD-LIBOR-BBA-3M',    '3M',  0.00425],
    ['USD-LIBOR-BBA-3M',    '6M',  0.00450],
]
df = pd.DataFrame(DUMMY_DATA, columns=['INDEX','AXIS1','VALUES'])

In [4]:
data_objs = []
for idx_name, sub in df.groupby("INDEX"):
    d1 = Data1D.createDataObject(
        data_type="zero_rate", 
        data_convention=idx_name,
        df=sub[["AXIS1", "VALUES"]]
    )
    data_objs.append(d1)

dc = DataCollection(data_objs)

## 3) Define build methods

Specify how each index should be interpolated; here we use piecewise constant interpolation.

In [5]:
build_methods = [
    {
      'TARGET':             'SOFR-1B',
      'DATA_TYPE':          'ZERO_RATE',
      'DATA_CONVENTION':    'SOFR-1B',
      'INTERPOLATION_METHOD':'PIECEWISE_CONSTANT'
    },
    {
      'TARGET':             'USD-LIBOR-BBA-3M',
      'DATA_TYPE':          'ZERO_RATE',
      'DATA_CONVENTION':    'USD-LIBOR-BBA-3M',
      'INTERPOLATION_METHOD':'PIECEWISE_CONSTANT'
    },
]


## 4) Instantiate the yield curve

Set the valuation (as‑of) date to June 26, 2025 and build the `YieldCurve` object.

In [6]:
as_of = "2025-06-26"
yc = YieldCurve(as_of, dc, build_methods)
print("YieldCurve built.")

YieldCurve built.


## 5) Compute and display spot discount factors

We pick several (index, date) pairs to show discount factors.

In [7]:
print("--- Spot Discount Factors ---")
tests = [
    ("SOFR-1B",          "2025-07-26"),
    ("SOFR-1B",          "2025-09-26"),
    ("USD-LIBOR-BBA-3M", "2025-07-26"),
    ("USD-LIBOR-BBA-3M", "2025-09-26"),
]
for idx, dt in tests:
    df_val = yc.discountFactor(idx, dt)
    print(f"{idx:>18} @ {dt}: DF = {df_val:.6f}")


--- Spot Discount Factors ---
           SOFR-1B @ 2025-07-26: DF = 0.999833
           SOFR-1B @ 2025-09-26: DF = 0.999420
  USD-LIBOR-BBA-3M @ 2025-07-26: DF = 0.999667
  USD-LIBOR-BBA-3M @ 2025-09-26: DF = 0.998960


## 6) Compute and display forward rates

Finally, we compute 1M and 3M overnight forwards, and a 3M LIBOR forward:

In [8]:
print("\n--- Forward Rates ---")
fwd_1m    = yc.forward("SOFR-1B",          as_of, "1M")
fwd_3m    = yc.forward("SOFR-1B",          as_of, "3M")
libor_fwd = yc.forward("USD-LIBOR-BBA-3M", as_of, "3M")

print(f"SOFR 1M forward:  {fwd_1m:.6%}")
print(f"SOFR 3M forward:  {fwd_3m:.6%}")
print(f"LIBOR 3M forward: {libor_fwd:.6%}")



--- Forward Rates ---
SOFR 1M forward:  0.200018%
SOFR 3M forward:  0.227240%
LIBOR 3M forward: 0.407277%
